Предполагаемый состав программного комплекса:
1. математическая модель котельного агрегата (имеется прототип, представленный в приложении);
2. уравнения, аппроксимирующие характеристики оборудования (имеется прототип, представленный в приложении);
3. база данных для хранения данных мониторинга (требуется разработка, на данный момент результаты экспериментов архивируются в csv файл и обрабатываются отдельно);
4. аналитический модуль (требуется разработка, на данный момент данные анализирует человек);
5. модуль дистанционного сбора данных (требуется разработка, на данный момент показания КИП напрямую считываются в компьютер, расположенный непосредственно у котлоагрегата и обрабатываются отдельно не в реальном времени);
6. интерфейс пользователя (прототип интерфейса представлен в приложении);

Основыне требования к ПО:
1. Возможность агрегирования экспериментальных данных в базе данных для их последующего использования в целях тестирования и верификации модели и статистической обработки данных в стороннем ПО.
2. Модульная структура ПО (например, замена модели котла или аппроксимирующих полиномов на другую с теми же входами/выходами не должна привести к прекращению работы интерфейса, базы данных и т.п.).
3. Дистанционная передача данных от КИП в модель должна быть реализована через сеть интернет (возможно, что программный модуль дистанционной передачи данных должен представлять из себя структуру: клиент (передача данных от КИП)-сервер-клиент(прием данных в модель))



Основные параметры (регуляторы): 
- расход топлива
- расход воды
- расход воздуха.

Основные выходные параметры (должны соответствовать режиму):
- соотношение топливо/воздух (альфа), безразм.;
- температура воды на выходе, оС; ------------> ГЛАВНОЕ
- температура УГ на выходе, оС;
- разряжение в топке (на данный момент не контролируется), Па.

Доп. параметры (в настоящее время принимаются константами):
- температура наружного воздуха, оС;
- температура воды на входе, оС;
- состав топлива (теплотворная способность топлива), кДж/кг.



Сама модель котла основана на расчете теплового баланса.

На данный момент имеется три полинома, полученные на основе обработки исторических данных:
- Tуг = f(Gтопл, Gдым.газ)  -- зависимость температуры уходящих газов от расхода топлива и расхода дымовых газов
- Tгор.воды = f(Gтопл, Gдым.газ) -- зависимость температуры горячей воды на выходе из котла от расхода топлива и расхода дымовых газов
- Tуг = f(Gтопл, Tгор.воды) -- зависимость температуры уходящих газов от расхода топлива и температуры горячей воды на выходе из котла
Эти полиномы описывают условно эталонные режимы работы котлоагрегата.

В модель котла на вход всегда подаются 4 величины: Gтопл, Gдым.газ, Tуг, Tгор.воды.
В зависимости от того, подаются фактические значения Tуг, Tгор.воды или значения из полиномов, модель выдает либо фактический КПД, либо теоретический КПД.

Сейчас могут быть реализованы следующие варианты расчета:
-  На вход модели подаются Gтопл, Gдым.газ, Tуг, Tгор.воды, полученные с измерительных приборов – в таком случае расчет ведется по данным эксперимента.

-  На вход модели подаются Gтопл, Gдым.газ, Tуг = f(Gтопл, Gдым.газ) , Tгор.воды = f(Gтопл, Gдым.газ),  т.е. только значения исключительно регулирующих воздействий – в таком случае расчет ведется только по эталонной модели.

-  На вход модели подаются Gтопл, Gдым.газ, Tуг = f(Gтопл, Tгор.воды), Tгор.воды, т.е. дополнительно вводится фактическое значение  Tгор.воды для более точного мониторинга Tуг (это позволяет реализовать «отслеживание» переходного процесса, что невозможно сделать чисто дискретной моделью из варианта б.) при этом Tгор.воды не должно сильно отклоняться от Tгор.воды = f(Gтопл, Gдым.газ), а Tуг не должно сильно отклоняться от Tуг = f(Gтопл, Gдым. газ), если отклонение велико, то где-то проблема с котлом.

Таким образом, чтобы произвести сравнение КПД и тепловой нагрузки (Qка), по модели нужно посчитать два раза – один раз по варианту а. и ещё один раз по варианту б. или в. Можно также посчитать три раза по всем вариантам а., б., в. и анализировать три значения.


In [ ]:
# Подключаем Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Аутентификация и создание PyDrive клиента
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Выводим список файлов в папке
folder_id = '1fhNsmhPUq7Xz1lAAu2hgyv0iJivR3tHM'
file_list = drive.ListFile({'q': "'{}' in parents".format(folder_id)}).GetList()
for item in file_list:
  print('title: {}, id: {}, mimeType: {}'.format(item['title'], item['id'], item['mimeType']))

title: 2020-09-17 Подготовленные для импорта в матлаб данные.xlsx, id: 1xtwh_4yNJSpr1pNtd8qoeoUYO3b7K-gtwSSpuTIiJis, mimeType: application/vnd.google-apps.spreadsheet
title: выгрузка данных с эксперимента 17.09.2020, id: 146VqxspNGOxk5iWPhCwzsG5wM_i2f9vLMf9kpi0EdnY, mimeType: application/vnd.google-apps.document
title: narod_attachment_links.html, id: 1IHBvQocl2T0-8qYCX94XKEQN2BTulm4i, mimeType: text/html


In [ ]:
# получаем все файлы в списке
files = list()
def get_dict(lst):
  for file1 in file_list:
    files_dict = dict()
    files_dict['title'] = file1['title']
    files_dict['id'] = file1['id']
    lst.append(files_dict)
    print(files_dict)
    train_downloaded = drive.CreateFile({'id': file1['id']})
    train_downloaded.GetContentFile(file1['title'], mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    
  return

In [ ]:
get_dict(files)
!ls

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
df_poly = pd.read_excel('2020-09-17 Подготовленные для импорта в матлаб данные.xlsx', usecols='E:K')
df_poly.head()

def fit_polynom(X, y): # апроксимация полиномом для двух переменных
    poly = PolynomialFeatures(degree=2)
    regression = LinearRegression()
    poly_variables = poly.fit_transform(X)
    model = regression.fit(poly_variables, y)
    return poly, model

def from_poly(model_type, X_new, df=df_poly):  # Определение температур на основе расходов (исторические данные) 
    """model_type выбирается из [t_gasmodel, t_gasmodelGtTwater, t_waterGtGgmodel]""" # различные варианты расчетов
    
    if model_type == 't_gasmodel':
        X = df[['Gtopl_kg_s - топливо, кг/с','Ggas_kgs - газы, кг/с']]
        y = df['Tgas - температура уходящих']
    
    elif model_type == 't_gasmodelGtTwater':
        X = df[['Gtopl_kg_s - топливо, кг/с','Twater - температура воды на выходе']]
        y = df['Tgas - температура уходящих']
    
    elif model_type == 't_waterGtGgmodel':
        X = df[['Gtopl_kg_s - топливо, кг/с','Ggas_kgs - газы, кг/с']]
        y = df['Twater - температура воды на выходе']
    
    poly, model = fit_polynom(X, y)
    X_new = np.array(X_new).reshape(1,-1)
    return model.predict(poly.transform(X_new))
    
from_poly('t_gasmodel', [0.000939, 0.004665])

array([151.96663345])

In [ ]:
def gas_properties(T, gas_name): # функции для расчета теплоемкости газов
    
    # примерные зависимости теплоемкости от температуры для газов, ККАЛ/м.куб
    if gas_name == "N2":  #азот
        cp_gas = -0.0000000062*T**3 + 0.0000409223*T**2 + 0.2967088606*T +0.9443609023
 
    elif gas_name == "O2": #кислород
        cp_gas = -0.0000000062*T**3 + 0.0000401636*T**2 + 0.320220539*T - 1.8166780588
    
    elif gas_name == "H2O": #водяной пар
        cp_gas = -0.0000000095*T**3 + 0.0000866913*T**2 + 0.3318386178*T +2.8430622010
     
    elif gas_name == "VOZD":  #воздух
        cp_gas = -0.0000000069*T**3 + 0.0000437430*T**2 + 0.3059695761*T +0.5648667122

    elif gas_name == "CO2":  #углекислоого газа
        cp_gas = -0.0000000211*T**3 + 0.0001135455*T**2 + 0.4394309172*T - 6.7252221463
        
    return cp_gas

In [ ]:
def kotel(g_topl_kgs, g_gas_kgs, t_gas, t_water): # Основная функция для расчета котла

    # 
    # -------  КОНСТАНТЫ  -------
    g_vod = 0.05  # Расход воды [кг/с]
    (Wp, A, Sp, C, H, N, O) = (7.2, 1.3, 0, 42.2, 5.6, 0.4, 43.3)  # Состав топлива
    Q_rn = 17192  # Теплота сгорания пеллет [кДж/кг]
    T_hol_vod = 15  # Температура холодной воды [oC]
    T_hvozd = 20  # Температура окружающего воздуха [oC]
    Cc_topl = 0.25  # теплоемкость сухой массы топлива [ккал/кг]
    T_topl = 20  # температура топлива [oC]
    ayn = 0.2  # доля уноса золы топлива (камерные топки, ТШУ, Кам уголь, в книге 0,95)

    # потери:
    q3 = 0.3  # от химической неполноты сгорания
    q4 = 1.5  # от механической неполноты сгорания
    q5 = 2  # теплоты через ограждение

    # потеря тепла с теплом шлака:
    ashl = 1 - ayn #  доля шлака:
    t_shl = 600 #  температура шлака при ТШУ [oC]
    I_shl = 560 #  энтальпия шлака:
    # -------------------

    #Расчет теоретического объема и состава уходящих газов на единицу топлива 

    # теоретическое количество сухого воздуха, необходимое для полного сгорания топлива:
    V_0 = 0.0889*(C + 0.375*Sp)+0.265*H - 0.0333*O  # [м3/кг.топлива]
    # теоретический объем водяных паров:
    V_h2o_0 = 0.111*H + 0.0124*Wp + 0.0161*V_0  # [м3/кг.топлива]
    # теоретический объем азота:
    V_n2_0 = 0.79*V_0 + 0.8*N/100  # [м3/кг.топлива]
    # объем трехатомных газов:
    V_ro2 = 1.866*(C + 0.375*Sp)/100  # [м3/кг.топлива]
    # теоретический объем дымовых газов:
    V_r_0 = V_h2o_0 + V_ro2 + V_n2_0  # [м3/кг.топлива]

    # Расчет альфа (избытка воздуха)
    rho_gas =  1.15 - 0.00202*t_gas;  # примерная аппроксимация плотности дымовых газов 
    V_g = (g_gas_kgs/rho_gas)/g_topl_kgs;  # [м.куб/кг.топл] реальный объемный расход дымовых газов
    alfa_t = (V_g - V_h2o_0 - V_ro2 - V_n2_0)/V_0 + 1;

    # Расчет энтальпий:

    # энтальпии воздуха, необходимого для полного сгорания топлива (?=1):
    I0_hvozd = gas_properties(T_hvozd, "VOZD")*V_0*4.187; # [кДж/кг.топлива]
    # тепло внесенное поступающим в котельный агрегат воздухом:
    Q_vvn = alfa_t*gas_properties(T_hvozd, "VOZD")*V_0*4.187;

    # тепло внесенное в котельный агрегат топливом:
    C_topl = Wp/100 + Cc_topl*(100 - Wp)/100  # теплоемкость рабочей массы топлива
    I_topl = C_topl*T_topl*4.187  # энтальпия топлива [кДж/кг.топлива]
    Qpp = Q_rn + Q_vvn + I_topl  # Располагаемое тепло на 1 кг топлива [кДж/кг.топлива]

    # Энтальпия газов:
    IGO = 4.187*(V_ro2*gas_properties(t_gas, "CO2") + 
                 V_n2_0*gas_properties(t_gas, "N2") + 
                 V_h2o_0*gas_properties(t_gas, "H2O")) # кДж/кг.топлива

    # 3. Потеря тепла с уходящими газами:
    # энтальпия уходящих газов:
    I_Guh1 = IGO + 4.187*(alfa_t - 1)*(V_0*gas_properties(t_gas, "VOZD"))  # 1!!!в примере не было коэффициент 4,187

    # Расчет потерь теплоты
    # приведенная величина уноса золы из топки:
    ayn_priv = 4.187*ayn*A/(Q_rn/1000)  # зачем она??

    q6 = ashl*I_shl*A/Qpp # потери со шлаком
    q2 = (I_Guh1 - alfa_t*I0_hvozd)*(100-q4)/Qpp # Потери теплоты с уходящими газами 

    q_summ = q2 + q3 + q4 + q5 + q6 # Суммарная потеря в котельном агрегате

    KPD_brutto = 100 - q_summ # КПД брутто по обратному балансу

    # Полное количество тепла, полезно отданного в котлоагрегате:
    Qka = g_topl_kgs*Qpp*(KPD_brutto/100)  # тепловая мощность котла кВт
    Qvod = g_vod*4.19*(t_water - T_hol_vod) # Тепловосприятие водой [кДж]
    KPDvod = (Qvod/Qka)*100 # КПД по воде (по прямому балансу)

    # Сопоставление балансов (сравнение КПД)
    delta = KPD_brutto - KPDvod
    
    return KPD_brutto, Qka, q2, g_vod, alfa_t, delta

In [ ]:
def calc_kotel(g_topl, g_gas, t_gas, t_water, calculation_type):
    """
    Расчет котла. Необходимо задать: g_topl, g_gas, t_gas, t_water,
    calculation_type -- 'real' or 'etalon1/etalon2 (???)"""
    
    # для сравнения температур (реальной и эталонной) ----------------
    # прогноз Т_уг по расходу топлива и воздуха
    Tgas_predict = from_poly('t_gasmodel', [g_topl, g_gas])[0]

    # прогноз Т_уг по расходу топлива и температуре воды на выходе
    Tgas_predict_TgasmodelGtTwater = from_poly('t_gasmodelGtTwater', [g_topl, t_water])[0]

    # прогноз Т_воды по расходу топлива и воздуха
    Twater_predict = from_poly('t_waterGtGgmodel', [g_topl, g_gas])[0]

    # прогноз Т_уг по расходу топлива и прогнозной температуре воды
    Tgas_predict_watermodel = from_poly('t_gasmodelGtTwater', 
                                        [g_topl, from_poly('t_waterGtGgmodel', [g_topl, g_gas])])[0]
    # ----------------------------------------------
    # расчет на основе фактических измерений 
    if calculation_type == 'real':
        data = pd.DataFrame(list(kotel(g_topl, g_gas, t_gas, t_water)) + [t_gas, t_water],
                            index=['KPD_brutto', 'Qka', 'q2', 'g_vod', 'alfa_t', 'delta', 'T_gas', 'T_water']).T
    # расчет котла по эталонной модели (входные параметры - только g_topl и g_gas)
    elif calculation_type == 'etalon':
        data = pd.DataFrame(list(kotel(g_topl, g_gas, Tgas_predict_watermodel, Twater_predict)) + 
                            [Tgas_predict_watermodel, Twater_predict],
                            index=['KPD_brutto', 'Qka', 'q2', 'g_vod', 'alfa_t', 'delta', 'T_gas', 'T_water']).T
    # расчет котла по эталонной модели с учетом фактических значений температуры воды на выходе
    elif calculation_type == 'etalon_2':                 
        data = pd.DataFrame(list(kotel(g_topl, g_gas, Tgas_predict_TgasmodelGtTwater, t_water)) + 
                            [Tgas_predict_TgasmodelGtTwater, t_water],
                            index=['KPD_brutto', 'Qka', 'q2', 'g_vod', 'alfa_t', 'delta', 'T_gas', 'T_water']).T

    return data


In [ ]:
def check_regime(real, etalon): # сравнение фактического режима с эталонным
    var = (real-etalon)/etalon*100
    for col in var.columns:
        print('Параметр {} отличается от эталонного на {} процентов'.format(col, np.round(var[col].values.item(), 2)))
    return var

In [ ]:
real = calc_kotel(0.000939, 0.004665, 155.287, 75.945, 'real') # входные данные из интерфейса, выходные данные в интерфейс
etalon = calc_kotel(0.000939, 0.004665, 155.287, 75.945, 'etalon') # входные данные из интерфейса, выходные данные в интерфейс

var = check_regime(real, etalon) # --> в интерфейс (зеленый, желтый, красный)

Параметр KPD_brutto отличается от эталонного на -0.22 процентов
Параметр Qka отличается от эталонного на -0.21 процентов
Параметр q2 отличается от эталонного на 3.17 процентов
Параметр g_vod отличается от эталонного на 0.0 процентов
Параметр alfa_t отличается от эталонного на 0.89 процентов
Параметр delta отличается от эталонного на -19.42 процентов
Параметр T_gas отличается от эталонного на 2.11 процентов
Параметр T_water отличается от эталонного на 0.19 процентов


In [ ]:
var.head()

In [ ]:
real

,KPD_brutto,Qka,q2,g_vod,alfa_t,delta,T_gas,T_water
0,89.725091,14.628357,6.441365,0.05,1.363958,2.442716,155.287,75.945
